In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import pystan

import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook', font_scale=1.2)

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline

from IPython.display import display

In [2]:
import pymc3 as pm
import theano
import theano.tensor as T
from scipy.stats.stats import pearsonr

In [3]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [4]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

In [5]:
query = """
SELECT * FROM d_prescriptions_vaso
"""
res = pd.read_sql_query(query, con)
res.head()

,drug,drug_name_poe,drug_name_generic,route,k_psinpshrine,k_levophed,k_neosynpshrine,k_methyleneblue,k_dopamine,k_phenylpshrine,k_vasopressin,k_adrenaline,k_isoprenaline,k_terlipressin
0,DOPamine,DOPamine,DOPamine,IV,0,0,0,0,1,0,0,0,0,0
1,DOPamine,None,None,IV,0,0,0,0,1,0,0,0,0,0
2,DOPamine,None,None,IV DRIP,0,0,0,0,1,0,0,0,0,0
3,DopAmine,DopAmine,DopAmine,IV DRIP,0,0,0,0,1,0,0,0,0,0
4,DopAmine,None,None,IV,0,0,0,0,1,0,0,0,0,0


In [6]:
res

,drug,drug_name_poe,drug_name_generic,route,k_psinpshrine,k_levophed,k_neosynpshrine,k_methyleneblue,k_dopamine,k_phenylpshrine,k_vasopressin,k_adrenaline,k_isoprenaline,k_terlipressin
0,DOPamine,DOPamine,DOPamine,IV,0,0,0,0,1,0,0,0,0,0
1,DOPamine,None,None,IV,0,0,0,0,1,0,0,0,0,0
2,DOPamine,None,None,IV DRIP,0,0,0,0,1,0,0,0,0,0
3,DopAmine,DopAmine,DopAmine,IV DRIP,0,0,0,0,1,0,0,0,0,0
4,DopAmine,None,None,IV,0,0,0,0,1,0,0,0,0,0
5,DopAmine,None,None,IV DRIP,0,0,0,0,1,0,0,0,0,0
6,Dopamine,Dopamine,Aspirin,IV DRIP,0,0,0,0,1,0,0,0,0,0
7,Dopamine,Dopamine,DopAmine,IV DRIP,0,0,0,0,1,0,0,0,0,0
8,Dopamine,Dopamine,Dopamine HCl,IV DRIP,0,0,0,0,1,0,0,0,0,0
9,Dopamine HCl,None,None,IV,0,0,0,0,1,0,0,0,0,0
